Data Model, Cleaning & Preprocessing

In [21]:
import pandas as pd
import numpy as nb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

df = pd.read_excel("/content/uk_train_rides.xlsx")

print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())



            Transaction ID Date of Purchase Time of Purchase Purchase Type  \
0  da8a6ba8-b3dc-4677-b176       2023-12-08         12:41:11        Online   
1  b0cdd1b0-f214-4197-be53       2023-12-16         11:23:01       Station   
2  f3ba7a96-f713-40d9-9629       2023-12-19         19:51:27        Online   
3  b2471f11-4fe7-4c87-8ab4       2023-12-20         23:00:36       Station   
4  2be00b45-0762-485e-a7a3       2023-12-27         18:22:56        Online   

  Payment Method Railcard Ticket Class Ticket Type  Price  \
0    Contactless    Adult     Standard     Advance     43   
1    Credit Card    Adult     Standard     Advance     23   
2    Credit Card      NaN     Standard     Advance      3   
3    Credit Card      NaN     Standard     Advance     13   
4    Contactless      NaN     Standard     Advance     76   

       Departure Station    Arrival Destination Date of Journey  \
0      London Paddington  Liverpool Lime Street      2024-01-01   
1     London Kings Cross      

Data Cleaning

In [26]:
df_clean = df.copy()

print("Missing values before cleaning:")
print(df_clean.isnull().sum())

# replace NaN with "no card"
df_clean['Railcard'] = df_clean['Railcard'].fillna('No Card')

# combining weather and weather conditions
df_clean['Reason for Delay'] = df_clean['Reason for Delay'].replace('Weather', 'Weather Conditions')

# unifying format of signal failure
df_clean['Reason for Delay'] = df_clean['Reason for Delay'].replace('Signal failure', 'Signal Failure')

# combining staff related reasons
df_clean['Reason for Delay'] = df_clean['Reason for Delay'].replace(['Staffing', 'Staff Shortage'], 'Staff-related')

print("Missing values after cleaning:")
print(df_clean.isnull().sum())


# checking duplicate values
print(f"\nDuplicates found: {df_clean.duplicated().sum()}")
df_clean.drop_duplicates(subset=['Transaction ID'], keep='first', inplace=True)

df_clean.head()
df_clean.info()


Missing values before cleaning:
Transaction ID             0
Date of Purchase           0
Time of Purchase           0
Purchase Type              0
Payment Method             0
Railcard               20918
Ticket Class               0
Ticket Type                0
Price                      0
Departure Station          0
Arrival Destination        0
Date of Journey            0
Departure Time             0
Arrival Time               0
Actual Arrival Time     1880
Journey Status             0
Reason for Delay       27481
Refund Request             0
dtype: int64
Missing values after cleaning:
Transaction ID             0
Date of Purchase           0
Time of Purchase           0
Purchase Type              0
Payment Method             0
Railcard                   0
Ticket Class               0
Ticket Type                0
Price                      0
Departure Station          0
Arrival Destination        0
Date of Journey            0
Departure Time             0
Arrival Time             

# 4. Data Validation and Outlier Detection


In [27]:
# Check for negative prices
print(f"\nNegative prices: {(df_clean['Price'] < 0).sum()}")
df_clean = df_clean[df_clean['Price'] >= 0]

# Check for unrealistic prices (e.g., > £1000)
print(f"Prices over £1000: {(df_clean['Price'] > 1000).sum()}")


Negative prices: 0
Prices over £1000: 0


Create Derived Columns

In [28]:
#Add time-based features
df_clean['Purchase Day of Week'] = df_clean['Date of Purchase'].dt.day_name()
df_clean['Purchase Month'] = df_clean['Date of Purchase'].dt.month
df_clean['Purchase Year'] = df_clean['Date of Purchase'].dt.year
df_clean['Journey Day of Week'] = df_clean['Date of Journey'].dt.day_name()
df_clean['Journey Month'] = df_clean['Date of Journey'].dt.month
df_clean['Is Weekend'] = df_clean['Journey Day of Week'].isin(['Saturday', 'Sunday'])

# Calculate delay in minutes
def calculate_delay(row):
    if pd.notna(row['Arrival Time']) and pd.notna(row['Actual Arrival Time']):
        scheduled = datetime.combine(datetime.today(), row['Arrival Time'])
        actual = datetime.combine(datetime.today(), row['Actual Arrival Time'])
        delay = (actual - scheduled).total_seconds() / 60
        return delay
    return 0

df_clean['Delay_Minutes'] = df_clean.apply(calculate_delay, axis=1)

# Categorize time of day
def categorize_time(time_obj):
    if pd.isna(time_obj):
        return 'Unknown'
    hour = time_obj.hour
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_clean['Journey Time Category'] = df_clean['Departure Time'].apply(categorize_time)

# Calculate journey duration in minutes
def calculate_duration(row):
    if pd.notna(row['Departure Time']) and pd.notna(row['Actual Arrival Time']):
        departure = datetime.combine(datetime.today(), row['Departure Time'])
        arrival = datetime.combine(datetime.today(), row['Actual Arrival Time'])
        duration = (arrival - departure).total_seconds() / 60
        return duration if duration >= 0 else duration + 1440  # Handle next-day arrivals
    return None

df_clean['Journey Duration Minutes'] = df_clean.apply(calculate_duration, axis=1)

df_clean.info()
df_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Transaction ID            31653 non-null  object        
 1   Date of Purchase          31653 non-null  datetime64[ns]
 2   Time of Purchase          31653 non-null  object        
 3   Purchase Type             31653 non-null  object        
 4   Payment Method            31653 non-null  object        
 5   Railcard                  31653 non-null  object        
 6   Ticket Class              31653 non-null  object        
 7   Ticket Type               31653 non-null  object        
 8   Price                     31653 non-null  int64         
 9   Departure Station         31653 non-null  object        
 10  Arrival Destination       31653 non-null  object        
 11  Date of Journey           31653 non-null  datetime64[ns]
 12  Departure Time    

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,...,Refund Request,Purchase Day of Week,Purchase Month,Purchase Year,Journey Day of Week,Journey Month,Is Weekend,Delay_Minutes,Journey Time Category,Journey Duration Minutes
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,...,No,Friday,12,2023,Monday,1,False,0.0,Morning,150.0
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,...,No,Saturday,12,2023,Monday,1,False,5.0,Morning,115.0
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,No Card,Standard,Advance,3,Liverpool Lime Street,...,No,Tuesday,12,2023,Tuesday,1,False,0.0,Evening,30.0
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,No Card,Standard,Advance,13,London Paddington,...,No,Wednesday,12,2023,Monday,1,False,0.0,Night,60.0
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,No Card,Standard,Advance,76,Liverpool Lime Street,...,No,Wednesday,12,2023,Monday,1,False,0.0,Afternoon,135.0


exporting

In [ ]:
df_clean.to_csv('cleaned_train_data.csv', index=False)
df_clean.to_excel('cleaned_train_data.xlsx', index=False)


